#last modified (yyyy/mm/dd): 2024/08/22

Author: Alessandro Ulivi (ale.ulivi@gmail.com)

The present pipeline allows to perform a semantic segmentation of 2 channels. It was conceptualized for PIP2::mCherry, CYK1::GFP. Refer to the ACR004 transgenic C. elegans line for background information on the transgenes.

The pipeline have been developed and tested on time-lapse images of the C.elegans embryo cortex. The embryo was imaged between the 2 and the 6 cells stages. Images were acquired, live, at a Nikon CSU-X1 spinning disk microscope at IGMBC Strasbourg, using a 100x, 1.4 NA, oil immersion objective (pixel size xy 0.11 um).

The pipeline works on 3D arrays. It was conceptualized for processing a single plane (the embryonic cortical plane) imaged over time. Thus, the segmentation masks are obtained by iterating 2D segmentations methods along one of the 3 axes of the input arrays. When the pipeline was created this corresponded to the time-dimension of the time-lapses.

When writing, the following channel-structure link was in place:
- channel 1 -> CYK1::GFP
- channel 2 -> PIP2::mCherry

To properly work the pipeline requires an existing input folder (input_folder) containg the two 3D arrays to process. The dimensions of the two arrays must match. The file-names of the two arrays must allow their unequivocal identification. This means that it is possible to have additional files in the input folder, as long as their names don't interfere with the identification of the two arrays to process (e.g. same file with different extension). The pipeline was conceptualized so that the two arrays correspond to the time-lapses for the two aforementioned channels.

When an ROI is used for segmentation, the iteration axis must be on position 0 of the input file shape. For example, when the pipeline was created this corresponded to the time-axis and the files had dimensions order TYX. Different dimension orders are NOT possible at the moment if a ROI is provided for segmentation because the reference image passed to form_mask_from_roi when creating embryo_roi_mask is at position 0 of the ch_timecourse.

It is required to provide an output folder. The output folder must exist. The output folder can contain files. The outputs of the pipeline are saved in the output folder.

The output of the pipeline are:
1) Binary segmentation mask for ch1 3D array. Refer to section SEGMENTATION OF CH1 - CYK1GFP ENRICHED STRUCTURES.
2) Binary segmentation mask for ch2 3D array. Refer to section SEGMENTATION OF CH2 - PIP2MCHERRY ENRICHED MEMBRANE DOMAINS.
3) A second binary segmentation mask for one of the two channels' 3D array. When the pipeline was created this corresponded to the segmentation of the emrbyo cortex using a ch2 (PIP2::mCherry). Refer to section SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY).


When creating output 3, it is possible to restrict the segmentation to one or more regions of interest (ROIs) of the input 3D array. In this case, the ROIs must be provided as a single file (.roi or .zip) contained in a folder (following roi_path variable). Nothing else but the file must be present in the folder. The procedure is conceptualized for the use of ROI files created in ImageJ/Fiji.



The following cell import packages and functions required for the pipeline.

The cell should be run.

The cell should not be modified.

In [69]:
#Import packages
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from utils import listdirNHF, form_mask_from_roi
from segment_pip2mcherry_cyk1gfp import organize_2_channels_files, mask_embryo, segment_pip2_enriched_domains, segment_cyk1_enriched_domains



DEFINE INPUT AND OUTPUT FOLDERS - DEFINE COMMON VARIABLES FOR DIFFERENT PROCESSING PARTS.

The following cell must be run.

Some parts of the following cell must be changed according to the processing to do.

In [95]:
#indicate the directories of the input_folder and the output_folder.
input_folder = r"/Users/ulivia/Desktop/Alessandro/projects/filopodia_paper/fig1_actin_nucleators/c_cyk1_pip2/step_03/200610-GM-ACR004/200610-GM-ACR004-0"
output_folder = r"/Users/ulivia/Desktop/Alessandro/projects/filopodia_paper/fig1_actin_nucleators/c_cyk1_pip2/step_04/200610-GM-ACR004/200610-GM-ACR004-0"

#indicate the target names for the imaged channels - these strings can be the full file names or part of them, but must allow to unequivocally link each channel to the respective
#file in the input_folder
ch1_target_name = "GFP" #Note: when the pipeline was created, channel-1 was cyk1::gfp signal
ch2_target_name = "RFP" #Note: when the pipeline was created, channel-2 was pip2:mcherry signal

#indicate saving names for the channels - indicate the name without extension - the present names  will be included in the names of the output segmentation masks of each channel
ch1_sav_ing_name = "cyk1_enriched_domains" #Note: when the pipeline was created, channel-1 was cyk1::gfp signal
ch2_sav_ing_name = "pip2_enriched_domains" #Note: when the pipeline was created, channel-3 was pip2::mcherry signal
embryo_mask_sav_ing_name = "embryo_mask"

#indicate in which dimension to iterate the 2D segmentation methods - when the pipeline was created this axis corresponded to time in the time-lapse image - when input files are created using
#open_rearrange_nikon_files this dimension is 0
time_axis = 0 #if ROI is provided for segmentation, this can only be 0

#Indicate if the file should be reduced of 1 pixel as it has been obtained with the splitting method based on Fiji
reduce_1_pixel= True
axis_2_reduce = 2

#Organize input files in a dictionary linking the channels' target names to their relative files - don't modify the line below
ch_names_linking_dict = organize_2_channels_files(input_folder, ch1_target_name, ch2_target_name, reduce_1_pixel=reduce_1_pixel, axis_2_reduce=axis_2_reduce)


In [96]:
for c in list(ch_names_linking_dict)[:2]:
    print(ch_names_linking_dict[c].shape)

(56, 928, 684)
(56, 928, 684)


SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)

Run the following 2 cells to obtain and save a binary segmentation of a channel time-lapse. When the pipeline was created this segmentation allowed to segment the embryo cortex from the backgound using the signal from channel 2, corresponding to pip2::mCherry.

To do the segmentation, the next two cells must be run. The first of the next two cells could be modified. The second of the next two cells should not be modified.





The segmentation follows the following steps:
1) Each 2D array (i) along the iteration axis of input array is smoothed using a gaussian kernel of size 5. Output name GAU(i).
2) The histogram distribution of intensity values is calculated (bins=100) per each GAU(i) along the iteration axis of input array. Output name HIS(i).
3) Per each HIS(i) along the iteration axis of input array, the position on the x axis of the mode value for the histogram distribution is calculated. Output name MOD(i).
4) Per each HIS(i) along the iteration axis of input array, an intensity value IV1(i) is calculated. IV1(i) corresponds to the minimum of the HIS(i) when only values higher than MOD(i) are considered.
5) Per each HIS(i) along the iteration axis of input array, an intensity value IV2(i) is calculated. IV2(i) corresponds to the maximum of HIS(i) when only values higher than MOD(i) are considered.
6) Per each HIS(i) along the iteration axis of input array, an intensity value IV3(i) is calculated. IV3(i) corresponds to the minimum of HIS(i) when only values higher than MOD(i) and smaller than IV2(i) are considered.
7) IV1 and IV2 are pooled for all 2D arrays along the iteraton axis. Output name POOL.
8) The median M and standard deviation S of POOL are calculated.
9) An intensity value THRESHOLD(i) is calculated per each 2D array (i) along the iteration axis of input array. THRESHOLD(i) corresponds to IV1(i) if IV1(i) is within 1 S from M. Otherwise, THRESHOLD(i) corresponds to IV2(i) if IV2(i) is within 1 S from M. Otherwise, THRESHOLD(i) corresponds to M.
10) Each gaussian smoothed 2D array GAU(i) along the iteraton axis is binarized using THRESHOLD(i) as highpass filter. Pixels whose intensity values are >THRESHOLD(i) are set to positive  Output THRESH_IMG(i).
11) Individual regions of THRESH_IMG(i) are groups of connected pixels with the same intensity value and completely sourranded by backgroung pixels. Refer to https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.label . Individual regions of each 2D THRESH_IMG(i) along the iteration axis are filered using a highpass filter. The value of the highpass filter (in number of pixels) is defined in the variable embryo_highpass_area_threshold. Only regions whose area is >embryo_highpass_area_threshold are maintained. Output FINAL_THRESH_IMG(i).


In [97]:
#THESE PARAMETERS COULD BE CHANGED TO REFINE THE SEGMENTATION

#Highpass filter area - most likely the following line should not be modified
embryo_highpass_area_threshold = 250 #Segmented structures smaller than this value (the unit is number of pixels) are removed. Suggested value is 250.

#Indicate if an an roi should be used for the embryo segmentation
#NOTE:
# 1) the roi is meant to restrict the segmentation procedure to a part of the time-course. This means that regions outside the provided roi will be automatically
# excluded from the segmentation result. This could be useful if, for example, there are bright structures near the target segmentation structure (supposedly the embryo cortex)
# which might interfer with the segmentation.
# 2) if use_roi_for_segmentation is set to True, the folder where the roi is stored must be indicated in roi_path
use_roi_for_segmentation = False
roi_path = r"/Users/ulivia/Desktop/Alessandro/projects/filopodia_paper/fig1_actin_nucleators/c_cyk1_pip2/step_03/200601-GM-ACR004_roi/200601-GM-ACR004-0" #Ignore this line if use_roi_for_segmentation=False



In [98]:
#GET (EMBRYO CORTEX) SEGMENTATION MASK FOR ALL TIMEPOINTS - THE PRESENT CELL SHOULD NOT BE MODIFIED

#Get ch2 timecourse - pip2-mCherry
ch2_timecourse = ch_names_linking_dict[ch2_target_name]

#Get the roi mask for segmentation if provided
if use_roi_for_segmentation:
    embryo_roi_file_path = os.path.join(roi_path, listdirNHF(roi_path)[0])
    embryo_roi_mask = form_mask_from_roi(embryo_roi_file_path, ch2_timecourse[0]) #this is the reason why the input file dimension must be TYX at the moment


# #Use mask_embryo on ch2 (pip2-mCherry, cortex plane) to get the mask of the embryo.
if use_roi_for_segmentation:
    embryo_thresholded_timecourse = mask_embryo(ch2_timecourse,
                                                area_threshold4embryo=embryo_highpass_area_threshold,
                                                sigma_gaussian_smt=5,
                                                int_hist_bins=100,
                                                roi_mask=embryo_roi_mask,
                                                time_axis=time_axis,
                                                output_lowval=0,
                                                output_highval=255,
                                                output_dtype=np.uint8)
else:
    embryo_thresholded_timecourse = mask_embryo(ch2_timecourse,
                                                area_threshold4embryo=embryo_highpass_area_threshold,
                                                sigma_gaussian_smt=5,
                                                int_hist_bins=100,
                                                time_axis=time_axis,
                                                output_lowval=0,
                                                output_highval=255,
                                                output_dtype=np.uint8)

#Save the result with the correct data structure
embryo_thresholded_timecourse_saving_name = ch_names_linking_dict['prefix_name'] + embryo_mask_sav_ing_name + ".ome.tif"

if time_axis==0:
    tifffile.imwrite(os.path.join(output_folder,embryo_thresholded_timecourse_saving_name), embryo_thresholded_timecourse, photometric='minisblack', metadata={"axes":'TYX'})
elif time_axis==1:
    tifffile.imwrite(os.path.join(output_folder,embryo_thresholded_timecourse_saving_name), embryo_thresholded_timecourse, photometric='minisblack', metadata={"axes":'YTX'})
elif time_axis==2:
    tifffile.imwrite(os.path.join(output_folder,embryo_thresholded_timecourse_saving_name), embryo_thresholded_timecourse, photometric='minisblack', metadata={"axes":'TYX'})


SEGMENTATION OF CH2 - PIP2MCHERRY ENRICHED MEMBRANE DOMAINS

Run the following 2 cells to obtain and save a binary segmentation of ch2 time-lapse based on a complex combination of filters aimed at segmenting bright and extended/filament-like stryctures.

When the pipeline was created this segmentation allowed to segment the pip2-enriched domains from the backgound using the signal from pip2::mCherry.

IMPORTANT NOTE: the present segmentation requires a binary mask restricting the processing to a ROI. This binary mask is loaded in the next cell in a variable called embryo_segmented_timecourse. The binary mask must have the same shape of the array which undergoes the present segmentation process. Positive pixels in the binary mask are interpreted as pixels of interest. The code is built so that the segmentation output of the section "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" is used as the binary mask. This means that either the "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" segmentation has been run, or its output must be present in the output_folder. When the pipeline was written, the binary mask provided the shape of the embryo cortex during the time-lapse.

To do the present segmentation, the next two cells must be run. The first of the next two cells could be modified. The second of the next two cells should not be modified.





The segmentation follows the following steps:
I will name 2DR (i) the i-th 2D array of the array to process, along the iteration axis. I will name 2DM (i) the corresponding i-th 2D array of the binary mask array.
1) Each 2DR (i) is smoothed using a bilater filter. Output name BIL(i).
2) Each BIL(i) is filtered using a Frangi-filter. Output name FRA(i).
2) The histogram distribution of intensity values is calculated (bins=100) per each FRA(i). The histogram is calculated only on the pixels of FRA(i) corresponded by positive pixels in 2DM (i). Output name HIS1(i).
4) Per each HIS1(i), an intensity value IV1(i) is calculated. IV1(i) corresponds to a maxima of HIS1(i) starting from the top of the x axis (high intensity values). The position of this maxima is set by the following parameter frangi_filter_peak_position.
5) IV1 are pooled for all FRA(i) allong the iteraton axis. Output name POOL.
6) The median M and standard deviation S of POOL are calculated.
7) An intensity value THRESHOLD1(i) is calculated per each FRA(i). THRESHOLD1(i) corresponds to IV1(i) if IV1(i) is within 1 S from M. Otherwise, THRESHOLD1(i) corresponds to M.
10) Each FRA(i) is binarized using THRESHOLD1(i) as highpass filter. Output name BIN1(i).
11) Each 2DR (i) is smoothed using a gaussian kernel of size 5 pixels. Output name GAU(i).
12) The histogram distribution of intensity values is calculated (bins=100) per each GAU(i). The histogram is calculated only on the pixels of GAU(i) corresponded by positive pixels in 2DM (i). Output name HIS2(i).
13) Per each HIS2(i), an intensity value IV2(i) is calculated. IV2(i) corresponds to the intensity values for the percentile of HIS2(i) indicated by variable ch2_top_percentile_hyst_filt.
14) Per each HIS2(i), an intensity value IV3(i) is calculated. IV3(i) corresponds to the intensity values for the percentile of HIS2(i) indicated by variable ch2_bot_percentile_hyst_filt.
15) Per each HIS2(i), the mode MOD(i) and standard deviation STD(i) are calculated.
16) Per each HIS2(i), an intensity value IV4(i) is calculated. IV4(i) corresponds to the intensity values of HIS2(i) calculated using the formula: IV4(i)=MOD(i)+(N1*STD(i)). Where N1 is indicated by variable ch2_top_stdv_multfactor_hyst_filt.
17) Per each HIS2(i), an intensity value IV5(i) is calculated. IV5(i) corresponds to the intensity values of HIS2(i) calculated using the formula: IV5(i)=MOD(i)+(N2*STD(i)). Where N2 is indicated by variable ch2_bot_stdv_multfactor_hyst_filt.
18) An intensity value THRESHOLD2(i) is calculated per each GAU(i). THRESHOLD2(i) corresponds to the biggest value between IV2(i) and IV4(i).
19) An intensity value THRESHOLD3(i) is calculated per each GAU(i). THRESHOLD3(i) corresponds to IV5(i) if IV5(i) is higher than IV2(i). Otherwise, IV5(i) is the smallest value between IV3(i) and IV5(i).
20) Each GAU(i) is binarized using an hysteresis based method (ref https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.apply_hysteresis_threshold). THRESHOLD2(i) and THRESHOLD3(i) are used, respectively, as high and low values for the hysteresis thresholding process. Output name BIN2(i).
21) Each BIN2(i) is filered on the corresponding BIN1(i) following the present logic: each region (regionprops https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) of BIN2(i) is maintained if at least 1 of the corresponding pixels in BIN1(i) is positive. Output name BIN3(i).
22) Each BIN1(i) is filtered using the present logic:
    22A) Per each region R(j)(i) of BIN1(i) (regionprops https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) the centroid Centr(j)(i) is calculated.
    22B) The closest distance between each Centr(j)(i) and the non-positive pixels of 2DM (i) is calculated. Output name Dist(j)(i).
    22C) R(j)(i) is kept if Dist(j)(i) is > parameter highpass_distance_threshold, removed otherwise.
    Output name BIN4(i).
23) Each BIN3(i) is merged with the corresponding BIN4(i). Output BIN5(i).
24) Each BIN5(i) is filtered following the present logic: each region (regionprops https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) of BIN5(i) is kept if its area (in number of pixels) is higher than a highpass filter. The highpass filter is set by parameter highpass_area_threshold. Output name FINAL_2D.


In [99]:
#DEFINE PARAMETERS FOR SEGMENTATION

#Gaussian smoothing ch2 images (pip2-enriched domains) - the number changes between fractions of 1 and the smallest of the xy pixel sizes of the input image.
# The higher the number the less sharper (and more approximated) the segmentation
gau_smooth_sigma_ch2 = 5 #recommended 5

# Bilateral filtering ch2 (pip2-enriched domains) - bilateral filtering is a smoothing method which preserves edges. Because the pipeline was created to segment pip2-enriched domains, this step
# is meant to preserve filament-like structures before applying a Frangi filtering - the following parameters are most likely not to be changed.
# Refer to OpenCV documentation (https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html -
# https://docs.opencv.org/4.x/d4/d86/group__imgproc__filter.html#ga9d7064d478c95d60003cf839430737ed)
bil_smooth_diameter = 4
bil_smooth_sigma_color = 20 
bil_smooth_sigma_space = 3

# Define the strictness image binarization after Frangi-filtering for ch2 (pip2-enriched domains)
# Frangi filtering is a typical edge detection technique (https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.frangi).
# In the context of this pipeline it allows to sharpen filament-like structures. After sharpening, the structures are segmented using a highpass intensity filter. The highpass threshold
# is obtained at the intensity value corresponding to a maxima in the histogram distribution intensity values of the Frangi-filter image.
# The present value (frangi_filter_peak_position) defines where the maxima is found along the x axis of the histogram distribution of intensity values.
# It is recommended to use a negative value. Most likely it ranges between -1 and -4. Negative values are interpreted as values starting from the high positions in the x axis of
# the histogram distribution of intensity values. In other words, if frangi_filter_peak_position is set to -1, the intensity value corresponding to the first maxima encountered starting
# from the high values of the histogram distribution of intensity values, will be used as threshold. If frangi_filter_peak_position is set to -2, the the intensity value corresponding
# to the second maxima encountered starting from the high values of the histogram distribution, will be used as threshold. And so on...
# When frangi_filter_peak_position is negative, the lower its value, the less strict will be the detection of filament-like structures.
frangi_filter_peak_position = -3  #recommended -1 to -3

# Parameters regulating hysteresis based segmentation - refer to get_hysteresis_based_segmentation in image_segmentation.py for documentation and to description of the segmentation steps
# in the past cell. In general, the higher the percentages, the stricted the segmentation of ch2 structures.
ch2_top_percentile_hyst_filt = 99 #percentile to use for calculating the high value to be passed to hysteresis filtering - suggested 99
ch2_bot_percentile_hyst_filt = 98 #percentile to use for calculating the low value to be passed to hysteresis filtering - suggested 98
ch2_top_stdv_multfactor_hyst_filt = 3.9 #how many standard deviations to sum to the mode of the intensity distribution, for calculating the high value to be passed to hysteresis filtering - suggested 3.9
ch2_bot_stdv_multfactor_hyst_filt = 2.8 #how many standard deviations to sum to the mode of the intensity distribution, for calculating the low value to be passed to hysteresis filtering - suggested 2.8

#Highpass area threshold - regions smaller than this area will be removed from the segmentation mask
highpass_area_threshold = 5 #Indicate the threshold in pixels: smaller structures will be excluded.

#Highpass distance threshold - regions whose centroid is closer to the background of the binary mask supporting this segmentation (supposedly the emrbyo cortex mask obtained from
# the past segmentation) than this distance will be removed. The binary mask is interpreted as positive pixels are pixels of interest, 0 or negative pixels as the background.
highpass_distance_threshold = 5 #Indicate the threshold (this unfortunately is arbitrary units).




#OPEN CH2 TIME-LAPSE AND THE BINARY MASK SUPPORTING THE PRESENT SEGMENTATION (when the pipeline was created this is the output of the 
# "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" segmentation: the segmented embryo cortex)
#Don't modify the lines below.
#If "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" segmentation has been run, use the result
try:
    embryo_segmented_timecourse = embryo_thresholded_timecourse

#If "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" segmentation has not been run, open the file from the mask saved in the output folder
except:
    #Get a list of files in the output folder
    list_output_files = listdirNHF(output_folder)
    #Select file which contains the embryo segmentaton saving name
    embryo_segmentation_file_name = [f for f in list_output_files if embryo_mask_sav_ing_name in f][0]
    #Open the embryo segmentation timecourse
    embryo_segmented_timecourse = tifffile.imread(os.path.join(output_folder, embryo_segmentation_file_name))
    

    #Open ch2 timecourse
    ch2_timecourse = ch_names_linking_dict[ch2_target_name]



In [100]:
#Iterate processing on all the timelapse and save the resust - don't modify the present cell
ch2_segmented_timecourse, ch2_processing_steps = segment_pip2_enriched_domains(ch2_timecourse,
                                                            embryo_segmented_timecourse,
                                                            sigma_gaussian_smoothing=gau_smooth_sigma_ch2,
                                                            diameter_bilateral_smoothing=bil_smooth_diameter,
                                                            sigmacol_bilateral_smoothing=bil_smooth_sigma_color,
                                                            sigmaspac_bilateral_smoothing=bil_smooth_sigma_space,
                                                            toppercent_hyster=ch2_top_percentile_hyst_filt,
                                                            botpercent_hyster=ch2_bot_percentile_hyst_filt,
                                                            topstdv_hyster=ch2_top_stdv_multfactor_hyst_filt,
                                                            botstdv_hyster=ch2_bot_stdv_multfactor_hyst_filt,
                                                            peakposition_frangi=frangi_filter_peak_position,
                                                            area_highpass_threshold=highpass_area_threshold,
                                                            distance_highpass_threshold=highpass_distance_threshold,
                                                            axis_to_use=0,
                                                            return_processing_steps=True,
                                                            output_arr_lowval=0,
                                                            output_arr_highval=255,
                                                            output_arr_dtype=np.uint8,
                                                            sigmas=range(1, 5, 1),
                                                            alpha=1, beta=1,
                                                            gamma=1,
                                                            black_ridges=False)

#Save the result with the correct data structure
ch2_thresholded_timecourse_saving_name = ch_names_linking_dict['prefix_name'] + ch2_sav_ing_name + ".ome.tif"

if time_axis==0:
    tifffile.imwrite(os.path.join(output_folder,ch2_thresholded_timecourse_saving_name), ch2_segmented_timecourse, photometric='minisblack', metadata={"axes":'TYX'})
elif time_axis==1:
    tifffile.imwrite(os.path.join(output_folder,ch2_thresholded_timecourse_saving_name), ch2_segmented_timecourse, photometric='minisblack', metadata={"axes":'YTX'})
elif time_axis==2:
    tifffile.imwrite(os.path.join(output_folder,ch2_thresholded_timecourse_saving_name), ch2_segmented_timecourse, photometric='minisblack', metadata={"axes":'TYX'})




=== 7 / 56
=== 14 / 56
=== 21 / 56
=== 28 / 56
=== 35 / 56
=== 42 / 56
=== 49 / 56
=== finished initial filtering ===
=== 7 / 56
=== 14 / 56
=== 21 / 56
=== 28 / 56
=== 35 / 56
=== 42 / 56
=== 49 / 56


In [101]:
#BY UNCOMMENTING THE PRESENT CELL IT IS POSSIBLE TO INSPECT ALL THE STEPS OF THE ABOVE SEGMENTATION

# #Show output
# timepoint_2_show = 9 #Indicate the timepoint to visualize. Note that python starts counting timepoints from 0.

# subplot_rows = 7
# subplot_cols = 2

# fig1, ax1 = plt.subplots(subplot_rows,subplot_cols,figsize=(20,80))

# count = 0
# for r in range(subplot_rows):
#     for c in range(subplot_cols):
#         if count<len(ch2_processing_steps[timepoint_2_show]):
#             if count == 6:
#                 count=count+1
#             elif count == 10:
#                 count=count+1
#             else:
#                 ax1[r][c].imshow(ch2_processing_steps[timepoint_2_show][count])
#                 count=count+1



SEGMENTATION OF CH1 - CYK1GFP ENRICHED STRUCTURES

Run the following 2 cells to obtain and save a binary segmentation of channel 1 time-lapse based on a hysteresis filtering.

When the pipeline was created this segmentation allowed to segment the cyk1-enriched domains from the backgound using the signal from cyk1::gfp.

IMPORTANT NOTE: the present segmentation requires a binary mask restricting the processing to a ROI. This binary mask is loaded in the next cell in a variable called embryo_segmented_timecourse_1. The binary mask must have the same shape of the array which undergoes the present segmentation process. Positive pixels in the binary mask are interpreted as pixels of interest. The code is built so that the segmentation output of the section "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" is used as the binary mask. This means that either the "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" segmentation has been run, or its output must be present in the output_folder. When the pipeline was written, the binary mask provided the shape of the embryo cortex during the time-lapse.

To do the segmentation, the next two cells must be run. The first of the next two cells could be modified. The second of the next two cells should not be modified.





The segmentation follows the following steps:
I will name 2DR (i) the i-th 2D array of the array to process, along the iteration axis. I will name 2DM (i) the corresponding i-th 2D array of the binary mask array.
1) Each 2DR (i) is smoothed using a median filter. Output name MED(i).
2) The histogram distribution of intensity values is calculated (bins=100) per each MED(i). The histogram is calculated only on the pixels of MED(i) corresponded by positive pixels in 2DM (i). Output name HIS(i).
3) Per each HIS(i), an intensity value IV1(i) is calculated. IV1(i) corresponds to the intensity values for the percentile of HIS(i) indicated by variable ch1_top_percentile_hyst_filt.
4) Per each HIS(i), an intensity value IV2(i) is calculated. IV2(i) corresponds to the intensity values for the percentile of HIS(i) indicated by variable ch1_bot_percentile_hyst_filt.
5) Per each HIS(i), the mode MOD(i) and standard deviation STD(i) are calculated.
6) Per each HIS(i), an intensity value IV3(i) is calculated. IV3(i) corresponds to the intensity values of HIS(i) calculated using the formula: IV3(i)=MOD(i)+(N1*STD(i)). Where N1 is indicated by variable ch1_top_stdv_multfactor_hyst_filt.
7) Per each HIS(i), an intensity value IV4(i) is calculated. IV4(i) corresponds to the intensity values of HIS(i) calculated using the formula: IV4(i)=MOD(i)+(N2*STD(i)). Where N2 is indicated by variable ch1_bot_stdv_multfactor_hyst_filt.
8) An intensity value THRESHOLD1(i) is calculated per each MED(i). THRESHOLD1(i) corresponds to the biggest value between IV1(i) and IV3(i).
9) An intensity value THRESHOLD2(i) is calculated per each MED(i). THRESHOLD2(i) corresponds to the biggest value between IV2(i) and IV4(i).
10) Each MED(i) is binarized using an hysteresis based method (ref https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.apply_hysteresis_threshold). THRESHOLD1(i) and THRESHOLD2(i) are used, respectively, as high and low values for the hysteresis thresholding process. Output name FINAL_2D.


In [102]:
#DEFINE PARAMETERS FOR SEGMENTATION

# Parameters regulating hysteresis based segmentation - refer to get_hysteresis_based_segmentation in image_segmentation.py for documentation and to description of the segmentation steps
# in the past cell. In general, the higher the percentages, the stricted the segmentation of ch1 structures.
ch1_top_percentile_hyst_filt = 99 #percentile to use for calculating the high value to be passed to hysteresis filtering - suggested 99
ch1_bot_percentile_hyst_filt = 98.7 #percentile to use for calculating the low value to be passed to hysteresis filtering - suggested 98.7
ch1_top_stdv_multfactor_hyst_filt = 5 #how many standard deviations to sum to the mode of the intensity distribution, for calculating the high value to be passed to hysteresis filtering - suggested 5
ch1_bot_stdv_multfactor_hyst_filt = 4 #how many standard deviations to sum to the mode of the intensity distribution, for calculating the low value to be passed to hysteresis filtering - suggested 4




#OPEN CH1 TIME-LAPSE AND THE BINARY MASK SUPPORTING THE PRESENT SEGMENTATION (when the pipeline was created this is the output of the 
# "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" segmentation: the segmented embryo cortex)
#Don't modify the lines below.
#If "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" segmentation has been run, use the result
try:
    embryo_segmented_timecourse_1 = embryo_thresholded_timecourse

#If "SEGMENTATION OF EMBRYO CORTEX USING CH2 (PIP2MCHERRY)" segmentation has not been run, open the file from the mask saved in the output folder
except:
    #Get a list of files in the output folder
    list_output_files = listdirNHF(output_folder)
    #Select file which contains the embryo segmentaton saving name
    embryo_segmentation_file_name_1 = [f for f in list_output_files if embryo_mask_sav_ing_name in f][0]
    #Open the embryo segmentation timecourse
    embryo_segmented_timecourse_1 = tifffile.imread(os.path.join(output_folder, embryo_segmentation_file_name_1))


#Open ch1 timecourse
ch1_timecourse = ch_names_linking_dict[ch1_target_name]


In [103]:
#Iterate segmentation on the timecourse - don't modify the present cell
ch1_segmented_timecourse = segment_cyk1_enriched_domains(ch1_timecourse,
                                                         embryo_segmented_timecourse_1,
                                                         hyst_filt_low_percentil_e=ch1_bot_percentile_hyst_filt,
                                                         hyst_filt_high_percentil_e=ch1_top_percentile_hyst_filt,
                                                         hyst_filt_low_stdfacto_r=ch1_bot_stdv_multfactor_hyst_filt,
                                                         hyst_filt_high_stdfacto_r=ch1_top_stdv_multfactor_hyst_filt,
                                                         output_low_va_l=0,
                                                         output_high_va_l=255,
                                                         output_d_typ_e=np.uint8,
                                                         iteration_axi_s=0)

#Save the result with the correct data structure
ch1_thresholded_timecourse_saving_name = ch_names_linking_dict['prefix_name'] + ch1_sav_ing_name + ".ome.tif"

if time_axis==0:
    tifffile.imwrite(os.path.join(output_folder,ch1_thresholded_timecourse_saving_name), ch1_segmented_timecourse, photometric='minisblack', metadata={"axes":'TYX'})
elif time_axis==1:
    tifffile.imwrite(os.path.join(output_folder,ch1_thresholded_timecourse_saving_name), ch1_segmented_timecourse, photometric='minisblack', metadata={"axes":'YTX'})
elif time_axis==2:
    tifffile.imwrite(os.path.join(output_folder,ch1_thresholded_timecourse_saving_name), ch1_segmented_timecourse, photometric='minisblack', metadata={"axes":'TYX'})
